In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


# unzip
import zipfile, os, shutil

dataset = '/content/gdrive/My Drive/kaggle/subset.zip'
dst_path = '/content/subset'
dst_file = os.path.join(dst_path, 'subset.zip') 

if not os.path.exists(dst_path):
  os.makedirs(dst_path)   

shutil.copy(dataset, dst_file)

with zipfile.ZipFile(dst_file, 'r') as file:
  file.extractall(dst_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from tensorflow.keras.applications import DenseNet169

conv_base = DenseNet169(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

KeyboardInterrupt: ignored

In [4]:
!pip install efficientnet
!pip install iterative-stratification

In [5]:
import efficientnet.keras as efn 
conv_base =  efn.EfficientNetB3(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

Using TensorFlow backend.


44113920/44107200 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.applications import NASNetLarge

conv_base = NASNetLarge(weights='imagenet',
                  include_top=False,
                  input_shape=(331, 331, 3))

343613440/343610240 [==============================] - 6s 0us/step


In [6]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential() 
model.add(conv_base)
model.add(layers.Flatten()) 
model.add(layers.Dense(256, activation='relu') )
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(40, activation='softmax'))


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
flatten_2 (Flatten)          (None, 487872)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               124895488 
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 40)                10280     
Total params: 209,822,586
Trainable params: 209,625,918
Non-trainable params: 196,668
_________________________________________________________________


In [8]:
conv_base.trainable = False


In [10]:
import os 
import numpy as np 

from keras.preprocessing.image import ImageDataGenerator

train_dir = os.path.join(dst_path, 'train')
validation_dir = os.path.join(dst_path, 'validation')

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255
                                   )


train_generator = train_datagen.flow_from_directory(train_dir,
                                          target_size=(150, 150),
                                          
                                          class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                          target_size=(150, 150),
                                         
                                          class_mode='sparse')



Found 1967 images belonging to 40 classes.
Found 533 images belonging to 40 classes.


In [11]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (32, 150, 150, 3)
labels batch shape: (32,)


In [ ]:
for data_batch, labels_batch in validation_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (32, 150, 150, 3)
labels batch shape: (32,)


In [ ]:
labels_batch

array([14., 21., 38., 37.,  2.,  4., 28., 36., 15.,  5., 22.,  3., 38.,
        3., 14., 27., 18., 25., 32.,  3.], dtype=float32)

In [ ]:
'''
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    set_trainable = True
  if set_trainable:
    layer.trainable = True
  else:
    layer.trainable = False
'''

In [13]:

model.compile(optimizer=optimizers.RMSprop(lr=1e-5), ##hyperparameter
              loss='sparse_categorical_crossentropy', 
              metrics=['acc'])

history = model.fit(train_generator,
                    epochs=100,
                    batch_size = 32, 
                    validation_data=validation_generator
                    )

ValueError: ignored